In [115]:
import pandas as pd
df_train = pd.read_csv('train.csv')


In [116]:
df_test = pd.read_csv('test.csv')
df_test.shape

(93800, 19)

In [43]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 140700 entries, 0 to 140699
Data columns (total 20 columns):
 #   Column                                 Non-Null Count   Dtype  
---  ------                                 --------------   -----  
 0   id                                     140700 non-null  int64  
 1   Name                                   140700 non-null  object 
 2   Gender                                 140700 non-null  object 
 3   Age                                    140700 non-null  float64
 4   City                                   140700 non-null  object 
 5   Working Professional or Student        140700 non-null  object 
 6   Profession                             104070 non-null  object 
 7   Academic Pressure                      27897 non-null   float64
 8   Work Pressure                          112782 non-null  float64
 9   CGPA                                   27898 non-null   float64
 10  Study Satisfaction                     27897 non-null   

In [13]:
df_train.isnull().sum()

id                                            0
Name                                          0
Gender                                        0
Age                                           0
City                                          0
Working Professional or Student               0
Profession                                36630
Academic Pressure                        112803
Work Pressure                             27918
CGPA                                     112802
Study Satisfaction                       112803
Job Satisfaction                          27910
Sleep Duration                                0
Dietary Habits                                4
Degree                                        2
Have you ever had suicidal thoughts ?         0
Work/Study Hours                              0
Financial Stress                              4
Family History of Mental Illness              0
Depression                                    0
dtype: int64

In [30]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 93800 entries, 0 to 93799
Data columns (total 19 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   id                                     93800 non-null  int64  
 1   Name                                   93800 non-null  object 
 2   Gender                                 93800 non-null  object 
 3   Age                                    93800 non-null  float64
 4   City                                   93800 non-null  object 
 5   Working Professional or Student        93800 non-null  object 
 6   Profession                             69168 non-null  object 
 7   Academic Pressure                      18767 non-null  float64
 8   Work Pressure                          75022 non-null  float64
 9   CGPA                                   18766 non-null  float64
 10  Study Satisfaction                     18767 non-null  float64
 11  Jo

Imputation of training data

In [117]:
df_train['Financial Stress'] = df_train['Financial Stress'].fillna(df_train['Financial Stress'].mean())

df_train['Dietary Habits'] = df_train['Dietary Habits'].fillna(df_train['Dietary Habits'].mode()[0])

df_train['Degree'] = df_train['Degree'].fillna(df_train['Degree'].mode()[0])

In [118]:
df_train['Job Satisfaction'] = df_train['Job Satisfaction'].fillna(df_train['Job Satisfaction'].mean())
df_train['Work Pressure'] = df_train['Work Pressure'].fillna(df_train['Work Pressure'].mean())

In [119]:
df_train['Profession'] = df_train['Profession'].fillna(df_train['Profession'].mode()[0])

In [120]:
df_train = df_train.drop(['Academic Pressure','CGPA','Study Satisfaction'], axis = 1)

Imputation of testing data

In [121]:
df_test['Dietary Habits'] = df_test['Dietary Habits'].fillna(df_test['Dietary Habits'].mode()[0])

df_test['Degree'] = df_test['Degree'].fillna(df_test['Degree'].mode()[0])

df_test['Profession'] = df_test['Profession'].fillna(df_test['Profession'].mode()[0])

In [122]:
df_test['Job Satisfaction'] = df_test['Job Satisfaction'].fillna(df_test['Job Satisfaction'].mean())

df_test['Work Pressure'] = df_test['Work Pressure'].fillna(df_test['Work Pressure'].mean())

In [123]:
df_test = df_test.drop(['Academic Pressure','CGPA','Study Satisfaction'], axis = 1)

Feature Selection

In [30]:
df_copy = pd.DataFrame()  # Initialize empty DataFrame
category = []
for col in df_train.columns:
    if df_train[col].dtype in ['int64', 'float64']:  # Check data type
        df_copy[col] = df_train[col]  # Copy column to df_copy
    else:
        category.append(col)

In [39]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Compute correlation matrix
corr_matrix = df_copy.corr()

# Select correlation with target variable
target_var = 'Depression'
target_correlation = corr_matrix[target_var].drop(target_var)  # Drop self-correlation

print(target_correlation)  # Print correlation values

id                  0.003944
Age                -0.564671
Work Pressure       0.137823
Job Satisfaction   -0.107254
Work/Study Hours    0.191746
Financial Stress    0.227232
Name: Depression, dtype: float64


In [40]:
import pandas as pd
import numpy as np
from scipy.stats import f_oneway

# Define categorical independent variables and continuous target

target = 'Depression'
col = ['Name',
 'Gender',
 'City',
 'Working Professional or Student',
 'Profession',
 'Sleep Duration',
 'Dietary Habits',
 'Degree',
 'Have you ever had suicidal thoughts ?',
 'Family History of Mental Illness']

# Perform ANOVA for each categorical feature
anova_results = {}

for col in category:
    groups = [df_train[df_train[col] == category][target] for category in df_train[col].unique()]
    f_stat, p_value = f_oneway(*groups)
    anova_results[col] = {'F-Statistic': f_stat, 'p-value': p_value}

# Convert results to DataFrame
anova_df_train = pd.DataFrame(anova_results).T
print(anova_df_train.sort_values(by="F-Statistic", ascending=False))


                                        F-Statistic        p-value
Working Professional or Student        52362.234543   0.000000e+00
Have you ever had suicidal thoughts ?  19522.362537   0.000000e+00
Profession                               352.102208   0.000000e+00
Dietary Habits                           152.133774   0.000000e+00
Degree                                   127.398698   0.000000e+00
Family History of Mental Illness          38.322757   6.012406e-10
Sleep Duration                            35.452694  6.790627e-237
City                                      11.676960  2.682903e-175


In [124]:
df_train = df_train.drop(['id','Work Pressure','Job Satisfaction','Dietary Habits'], axis = 1)

In [125]:
df_train = df_train.drop(['Name','Gender','Family History of Mental Illness','City'], axis = 1)
df_train

,Age,Working Professional or Student,Profession,Sleep Duration,Degree,Have you ever had suicidal thoughts ?,Work/Study Hours,Financial Stress,Depression
0,49.0,Working Professional,Chef,More than 8 hours,BHM,No,1.0,2.0,0
1,26.0,Working Professional,Teacher,Less than 5 hours,LLB,Yes,7.0,3.0,1
2,33.0,Student,Teacher,5-6 hours,B.Pharm,Yes,3.0,1.0,1
3,22.0,Working Professional,Teacher,Less than 5 hours,BBA,Yes,10.0,1.0,1
4,30.0,Working Professional,Business Analyst,5-6 hours,BBA,Yes,9.0,4.0,0
...,...,...,...,...,...,...,...,...,...
140695,18.0,Working Professional,Teacher,5-6 hours,Class 12,No,2.0,4.0,1
140696,41.0,Working Professional,Content Writer,7-8 hours,B.Tech,Yes,6.0,5.0,0
140697,24.0,Working Professional,Marketing Manager,More than 8 hours,B.Com,No,4.0,4.0,0
140698,49.0,Working Professional,Plumber,5-6 hours,ME,Yes,10.0,1.0,0


In [126]:
df_test = df_test.drop(['id','Work Pressure','Job Satisfaction'], axis = 1)

In [127]:
df_test = df_test.drop(['Name','Gender','Family History of Mental Illness','City','Dietary Habits'], axis = 1)
df_test

,Age,Working Professional or Student,Profession,Sleep Duration,Degree,Have you ever had suicidal thoughts ?,Work/Study Hours,Financial Stress
0,53.0,Working Professional,Judge,Less than 5 hours,LLB,No,9.0,3.0
1,58.0,Working Professional,Educational Consultant,Less than 5 hours,B.Ed,No,6.0,4.0
2,53.0,Working Professional,Teacher,7-8 hours,B.Arch,Yes,12.0,4.0
3,23.0,Student,Teacher,More than 8 hours,BSc,Yes,10.0,4.0
4,47.0,Working Professional,Teacher,7-8 hours,BCA,Yes,3.0,4.0
...,...,...,...,...,...,...,...,...
93795,49.0,Working Professional,Pilot,Less than 5 hours,BSc,Yes,2.0,2.0
93796,29.0,Working Professional,Pilot,7-8 hours,BE,Yes,11.0,3.0
93797,24.0,Student,Teacher,7-8 hours,B.Tech,No,7.0,1.0
93798,23.0,Working Professional,Marketing Manager,5-6 hours,BA,Yes,7.0,5.0


# Data preprocessing pipeline

In [ ]:
import pickle
from sklearn.preprocessing import LabelEncoder

# Encoding 'Work/Study Hours'
le_work_study = LabelEncoder()
df_train['Work/Study Hours'] = le_work_study.fit_transform(df_train['Work/Study Hours'])

with open("Work_study_le.pkl", "wb") as f:
    pickle.dump(le_work_study, f)

# Encoding 'Sleep Duration'
le_sleep = LabelEncoder()
df_train['Sleep Duration'] = le_sleep.fit_transform(df_train['Sleep Duration'])

with open("Sleep_Duration_le.pkl", "wb") as f:
    pickle.dump(le_sleep, f)

# Encoding 'Financial Stress'
le_financial = LabelEncoder()
df_train['Financial Stress'] = le_financial.fit_transform(df_train['Financial Stress'])

with open("Financial_Stress_le.pkl", "wb") as f:
    pickle.dump(le_financial, f)

print("Label Encoders Saved!")


Label Encoders Saved!


In [128]:
def label_encoding(y,col):
    from sklearn import preprocessing

    label_encoder = preprocessing.LabelEncoder()
    y[col] = label_encoder.fit_transform(y[col])
    return y

label_encoding(df_train,'Work/Study Hours')
label_encoding(df_train,'Sleep Duration')
label_encoding(df_train,'Financial Stress')

label_encoding(df_test,'Work/Study Hours')
label_encoding(df_test,'Sleep Duration')
label_encoding(df_test,'Financial Stress')

,Age,Working Professional or Student,Profession,Sleep Duration,Degree,Have you ever had suicidal thoughts ?,Work/Study Hours,Financial Stress
0,53.0,Working Professional,Judge,25,LLB,No,9,2
1,58.0,Working Professional,Educational Consultant,25,B.Ed,No,6,3
2,53.0,Working Professional,Teacher,16,B.Arch,Yes,12,3
3,23.0,Student,Teacher,27,BSc,Yes,10,3
4,47.0,Working Professional,Teacher,16,BCA,Yes,3,3
...,...,...,...,...,...,...,...,...
93795,49.0,Working Professional,Pilot,25,BSc,Yes,2,1
93796,29.0,Working Professional,Pilot,16,BE,Yes,11,2
93797,24.0,Student,Teacher,16,B.Tech,No,7,0
93798,23.0,Working Professional,Marketing Manager,11,BA,Yes,7,4


In [108]:
df_profession = df_train['Profession']
df_degree = df_train['Degree']
df_working_or_student = df_train['Working Professional or Student']
df_suicidal_thoughts = df_train['Have you ever had suicidal thoughts ?']


In [114]:
import category_encoders as ce
import pickle

pro_encoder = ce.BinaryEncoder(cols=['Profession'])
df_encoded = pro_encoder.fit_transform(df_profession)  

deg_encoder = ce.BinaryEncoder(cols=['Degree'])
df_encoded = deg_encoder.fit_transform(df_degree) 

w_s_encoder = ce.BinaryEncoder(cols=['Working Professional or Student'])
df_encoded = w_s_encoder.fit_transform(df_working_or_student)  

thoughts_encoder = ce.BinaryEncoder(cols=['Have you ever had suicidal thoughts ?'])
df_encoded = thoughts_encoder.fit_transform(df_suicidal_thoughts)  


with open("pro_encoder.pkl", "wb") as f:
    pickle.dump(pro_encoder, f)

with open("deg_encoder.pkl", "wb") as f:
    pickle.dump(deg_encoder, f)

with open("w_s_encoder.pkl", "wb") as f:
    pickle.dump(w_s_encoder, f)

with open("thoughts_encoder.pkl", "wb") as f:
    pickle.dump(thoughts_encoder, f)           

print("Binary Encoder Saved!")

Binary Encoder Saved!


In [129]:
import category_encoders as ce
def binary_encoding(x_encoded,col):


    binary_encoder = ce.BinaryEncoder(cols = [col])
    x_encoded = binary_encoder.fit_transform(x_encoded)  # Creates a new DataFrame
    return x_encoded

df_train = binary_encoding(df_train,'Profession')
df_train = binary_encoding(df_train,'Degree')
df_train = binary_encoding(df_train,'Working Professional or Student')
df_train = binary_encoding(df_train,'Have you ever had suicidal thoughts ?')

df_test = binary_encoding(df_test,'Profession')
df_test = binary_encoding(df_test,'Degree')
df_test = binary_encoding(df_test,'Working Professional or Student')
df_test = binary_encoding(df_test,'Have you ever had suicidal thoughts ?')


In [130]:

from sklearn.preprocessing import StandardScaler

def scaling(x,col):

  scaler = StandardScaler()
  x[col] = scaler.fit_transform(x[[col]])
  return x
  
scaling(df_train,'Age')

scaling(df_test,'Age')
  

,Age,Working Professional or Student_0,Working Professional or Student_1,Profession_0,Profession_1,Profession_2,Profession_3,Profession_4,Profession_5,Profession_6,...,Degree_1,Degree_2,Degree_3,Degree_4,Degree_5,Degree_6,Have you ever had suicidal thoughts ?_0,Have you ever had suicidal thoughts ?_1,Work/Study Hours,Financial Stress
0,1.022988,0,1,0,0,0,0,0,0,1,...,0,0,0,0,0,1,0,1,9,2
1,1.426428,0,1,0,0,0,0,0,1,0,...,0,0,0,0,1,0,0,1,6,3
2,1.022988,0,1,0,0,0,0,0,1,1,...,0,0,0,0,1,1,1,0,12,3
3,-1.397652,1,0,0,0,0,0,0,1,1,...,0,0,0,1,0,0,1,0,10,3
4,0.538860,0,1,0,0,0,0,0,1,1,...,0,0,0,1,0,1,1,0,3,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93795,0.700236,0,1,0,0,1,0,0,0,0,...,0,0,0,1,0,0,1,0,2,1
93796,-0.913524,0,1,0,0,1,0,0,0,0,...,0,1,0,0,1,1,1,0,11,2
93797,-1.316964,1,0,0,0,0,0,0,1,1,...,0,1,0,1,0,1,0,1,7,0
93798,-1.397652,0,1,0,0,0,1,1,0,0,...,0,0,1,0,0,0,1,0,7,4


pickling

In [44]:
import pickle

with open("age_scaler.pkl", "wb") as f:
    pickle.dump(age_scaler, f)

print("Scaler saved successfully!")

Scaler saved successfully!


# Deep learning Classification model

In [100]:
import torch
import torch.nn as nn # Sub class for neural network
import torch.optim as optim # All the optimization algorithm Object creator
import torch.nn.functional as F 

In [101]:
X = df_train.drop('Depression', axis= 1)
y = df_train['Depression']

Over sampling 

In [134]:
from imblearn.over_sampling import SMOTE

smote = SMOTE(sampling_strategy="auto", random_state=42)

X_resampled, y_resampled = smote.fit_resample(X, y)

print("Resampled X shape:", X_resampled.shape)
print("Resampled y shape:", y_resampled.shape)


Resampled X shape: (230266, 22)
Resampled y shape: (230266,)


In [47]:
import torch

X_resampled = torch.tensor(X_resampled.values, dtype=torch.float32)
y_resampled = torch.tensor(y_resampled, dtype=torch.float32).reshape(-1, 1)

In [48]:
class classification(nn.Module):
    def __init__(self):
        super(classification,self).__init__()
        self.layer1 = nn.Linear(22,48)
        self.act1 = nn.ReLU()
        self.layer2 = nn.Linear(48, 32)
        self.act2 = nn.ReLU()
        self.layer3 = nn.Linear(32, 16)
        self.act3 = nn.ReLU()
        self.output = nn.Linear(16, 1)
        self.sigmoid = nn.Sigmoid()
 
    def forward(self, x):
        x = self.act1(self.layer1(x))
        x = self.act2(self.layer2(x))
        x = self.act3(self.layer3(x))
        x = self.sigmoid(self.output(x))
        return x

In [ ]:
import tqdm
import torch
import torch.nn as nn
import torch.optim as optim

def model_train(clf, X_resampled, y_resampled, n_epochs=25, batch_size=14070, lr=0.001):
  
    loss_function = nn.BCELoss()  # Binary Cross-Entropy Loss
    optimizer = optim.Adam(clf.parameters(), lr=lr)
    
    batch_start = torch.arange(0, len(X_resampled), batch_size)

    for epoch in range(n_epochs):
        clf.train()  # Set model to training mode
        total_loss = 0.0  

        with tqdm.tqdm(batch_start, unit="batch", mininterval=0, disable=False) as bar:
            bar.set_description(f"Epoch {epoch + 1}")

            for start in bar:
                # Take a batch
                X_batch = X_resampled[start:start + batch_size]
                y_batch = y_resampled[start:start + batch_size]

                # Forward pass
                y_pred = clf(X_batch)
                loss = loss_function(y_pred, y_batch)

                # Backward pass
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

                # Update loss
                total_loss += loss.item()
                bar.set_postfix(loss=loss.item())


        clf.eval()
        with torch.no_grad():  
            y_train_pred = clf(X_resampled)
            train_acc = (y_train_pred.round() == y_resampled).float().mean().item()  
        print(f"Epoch {epoch + 1}: Loss = {total_loss:.4f}, Training Accuracy = {train_acc:.4f}")

    return train_acc  


clf = classification()  
train_accuracy = model_train(clf, X_resampled, y_resampled)
print(f"Final Training Accuracy: {train_accuracy:.4f}")

Epoch 1: 100%|██████████| 17/17 [00:03<00:00,  5.62batch/s, loss=1.09] 


Epoch 1: Loss = 13.6216, Training Accuracy = 0.5000


Epoch 2: 100%|██████████| 17/17 [00:03<00:00,  4.83batch/s, loss=0.933]


Epoch 2: Loss = 12.3069, Training Accuracy = 0.5000


Epoch 3: 100%|██████████| 17/17 [00:03<00:00,  5.63batch/s, loss=0.833]


...

Epoch 23: Loss = 4.8566, Training Accuracy = 0.8993


Epoch 24: 100%|██████████| 17/17 [00:03<00:00,  5.11batch/s, loss=0.201]


Epoch 24: Loss = 4.8220, Training Accuracy = 0.8997


Epoch 25: 100%|██████████| 17/17 [00:02<00:00,  8.02batch/s, loss=0.2]  


Epoch 25: Loss = 4.7914, Training Accuracy = 0.9002
Final Training Accuracy: 0.9002


In [ ]:
# Convert X_train to PyTorch tensor (if not already)
X_train_tensor = X_resampled

# Set model to evaluation mode
clf.eval()

with torch.no_grad():
    y_train_pred = clf(X_train_tensor)  
y_train_pred_labels = (y_train_pred >= 0.5).float().squeeze().numpy()

In [52]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import torch

y_train_actual = y_resampled.squeeze().numpy()

# Compute evaluation metrics
accuracy = accuracy_score(y_train_actual, y_train_pred_labels)
precision = precision_score(y_train_actual, y_train_pred_labels)
recall = recall_score(y_train_actual, y_train_pred_labels)
f1 = f1_score(y_train_actual, y_train_pred_labels)

# Print results
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-score: {f1:.4f}")


Accuracy: 0.9002
Precision: 0.8712
Recall: 0.9391
F1-score: 0.9039


In [53]:
X_test = df_test
X_test = torch.tensor(X_test.values, dtype=torch.float32)

saving the model

In [54]:
import torch

torch.save(clf.state_dict(), "classification_weights.pth")
print("Trained model weights saved successfully!")


Trained model weights saved successfully!


loading the model

In [55]:
import torch
clf = classification()  

# Load saved model weights
clf.load_state_dict(torch.load("classification_weights.pth"))

# Set to evaluation mode (important for inference)
clf.eval()

print("Trained model loaded successfully!")


Trained model loaded successfully!


Testing unseen data

In [56]:
with torch.no_grad():  # Disable gradients for inference
    test_predictions = clf(X_test)  # Predict on test data
    test_predictions = test_predictions.round()  # Convert probabilities to 0/1 for classification

print("Predictions on unseen data:", test_predictions)


Predictions on unseen data: tensor([[0.],
        [0.],
        [0.],
        ...,
        [1.],
        [1.],
        [0.]])
